In [ ]:
# Import necessary packages
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

##### Extract one individual dataframe per type of data (questionnaire, FFQ, clinical measures and clinical events)

In [ ]:
# Import dictionary the complete Whitehall II dictionary
xls = pd.ExcelFile('../Whitehall/W2_data_dictionary_DPUK_v0.5.xlsx')

# Create multiple DataFrames of the different sheets in the Whitehall II dictionary
questionnaire = pd.read_excel(xls, 'Questionnaire')
FFQ = pd.read_excel(xls, 'FFQ')
clinical_measures = pd.read_excel(xls, 'Clinical_measures')
clinical_events = pd.read_excel(xls, 'Clinical_events')

# Create a list containing the dataframes that have phases 9 till 11 
df_list = [questionnaire, FFQ, clinical_measures, clinical_events]

In [ ]:
# A function that removes all features that were not collected in phase 9 and only keeps useful information
def subset_dataframes(dataframes, columns_to_keep):
    modified_dfs =[]
    for df in dataframes:
        # Remove the rows that miss all values for phase 9 
        df = df.dropna(subset=['PHASE9'])

        # Only keep specified columns 
        modified_df = df[columns_to_keep]

        # Add the dataframe to a list 
        modified_dfs.append(modified_df)
    return modified_dfs

modified_dfs = subset_dataframes(df_list, ['CATEGORY', 'SUBCATEGORY', 'DESCRIPTION', 'PHASE9'])

In [ ]:
# Give data frames in the modified_dfs list an interpretable name, and add the new names to a list
questionnaire_mod = modified_dfs[0]
FFQ_mod = modified_dfs[1]
clinical_measures_mod = modified_dfs[2]
clinical_events_mod = modified_dfs[3]

modified_df_list = [questionnaire_mod, FFQ_mod, clinical_measures_mod, clinical_events_mod]

In [ ]:
# A function that creates a dictionary where:
# Keys are the type of data (questionnaire, FFQ, clinical measures and clinical events)
# And the items corresponding to the keys are the column names 

def variables_to_dict(dataframes, key_names, column_name, result_dict):
    for df, key_name in zip(dataframes, key_names):
        result_dict[key_name] = df[column_name].to_list()

# Create a list of interpretable key names
key_names = ['questionnaire_fts', 'FFQ_fts', 'clinical_measures_fts', 'clinical_events_fts']
column_name = 'PHASE9'

# Create an empty dictionary that can be used to save the variable names per data type 
columns_whitehall = {}

variables_to_dict(modified_df_list, key_names, column_name, columns_whitehall)

print(columns_whitehall)

In [ ]:
# Save dictionary to access later 
with open('../Processed datasets/feature_dictionary.pkl', 'wb') as f:
    pickle.dump(columns_whitehall, f)

In [ ]:
# Load dictionary from files to use
with open('../Processed datasets/feature_dictionary.pkl', 'rb') as f:
    columns_whitehall = pickle.load(f)

#### Subset the data based on the selected features from phase 9

In [ ]:
# Import Whitehall II dataset
df = pd.read_csv('../Whitehall/0620_DATASET.csv')

In [ ]:
# Make a list of variables that are not from phase 9 but should be kept because they are necessary for creating the target variable
target_fts = ["KDIABYR", "FDRG0612", "FDRG6122","FDRG6120", 
              "FDRG6123", "FDRG6121", "FDRG0611",
              "FDRG6112", "FDRG6110", "FDRG6111",'FHBA']

In [ ]:
# Make a list of features that were collected in phase 9 
selected_columns = []
for key, columns in columns_whitehall.items():
    selected_columns.extend(columns)

In [ ]:
# Add all required features to one list to subset the complete Whitehall II dataset accordingly 
all_columns = ['Id_random_DPUK'] + target_fts + selected_columns 
df_complete = df[all_columns].copy()

#### Create one big dataframe with all the necessary data 

In [ ]:
# Import the separate data file that needs to be merged with the filtered Whitehall dataset
basics = pd.read_csv('../Whitehall/ADDITIONAL VARIABLES/0620v2_DATASET.csv')

In [ ]:
# Merge the features of phase 9 with the basic data of respondents 
total_df = pd.merge(df_complete, basics, on='Id_random_DPUK', how='inner')
total_df

In [ ]:
# Remove all the participants where Haemoglobine A-1C is unknown in phase 11
df_hgb = total_df[total_df["FHBA"].notna()].copy()

#### Remove the participants that did not participate in any of the data collection efforts of phase 9

In [ ]:
# Remove all the participants who did not participate at all in Phase 9
df_phase9 = df_hgb.loc[df_hgb['JPART'] == 1].copy()

#### Create outcome variable

In [ ]:
# Create a function that determines whether someone is diagnosed with diabetes before or after 18
def diagnosed_before_18(row):

    # Calculate the age of diagnosis 
    age_diagnosed8 = row["KDIABYR"] - 1900 - row['YOB_C']
    age_diagnosed9 = row["JDIABYR"] - 1900 - row['YOB_C']

    # Initialize the result as NaN
    result = np.nan

    # Determine if someone is diagnosed before 18 
    if age_diagnosed8 < 18:
        result = True
    elif age_diagnosed8 >= 18:
        result = False
    elif age_diagnosed9 < 18:
        result = True
    if age_diagnosed9 >= 18: # overrule age_diagnosed8 in case patient had developed diabetes later
        result = False

    return result

In [ ]:
# Create a function that determines whether a participant has Diabetes Type 1 
def only_takes_insulin(row):

    total_oral_tablets = 0
     
    oral_tablets_row = row[["JDRG0612", "FDRG0612",
                           "JDRG6122", "FDRG6122",
                           "JDRG6120", "FDRG6120",
                           "JDRG6123", "FDRG6123",
                           "JDRG6121", "FDRG6121"]].copy()
    
    insulin_row = row[["JDRG0611", "FDRG0611",
                    "JDRG6112", "FDRG6112",
                    "JDRG6110", "FDRG6110",
                    "JDRG6111", "FDRG6111"]].copy()

    # Check if someone uses oral diabetes tablets 
    if oral_tablets_row.sum() >= 1:
        return False
    # Check if someone uses insulin     
    elif insulin_row.sum() >= 1:
        return True
    # Return NaN if someone does not use oral tablets or insulin    
    else:
        return np.nan

In [ ]:
# Create a function that determines whether someone's haemoglobin A1C values are above the set 7% diabetes threshold 
# 0 means no diabetes
# 1 means diabetes type 1
# 2 means diabetes type 2 only in Phase 11
# 3 means diabetes type 2 only Phase 9 
# 4 means diabetes in Phases 9 and 11 

def high_hba1c(row):

    if row["JHBA"] <= 7 and row["FHBA"] <= 7:
        return 0
    elif row["JHBA"] <= 7 and row["FHBA"] > 7:
        return 2
    elif row["JHBA"] > 7 and row["FHBA"] <= 7:
        return 3 
    else:
        return 4 

In [ ]:
# Create a function that determines whether someone had type 2 diabetes in phase 9 and 11 
def takes_oral_tablets(row):

    oral_tablets_11 = row[["FDRG0612",
                           "FDRG6122",
                           "FDRG6120",
                           "FDRG6123",
                           "FDRG6121"]].copy()
    
    oral_tablets_9 = row[["JDRG0612",
                           "JDRG6122",
                           "JDRG6120",
                           "JDRG6123",
                           "JDRG6121"]].copy()

    # If someone uses oral diabetes medicine in phases 9 and 11, return 4
    if oral_tablets_11.sum() >= 1 and oral_tablets_9.sum() >= 1:
        return 4

    # If someone only uses medicine in phase 9, return 3
    elif oral_tablets_9.sum() >= 1:
        return 3

    # If someone only uses medicine in phase 11, return 2 
    elif oral_tablets_11.sum() >= 1:
        return 2
    else:
        return np.nan

In [ ]:
# Create a function that determines the target where:
# 0 means not diabetic
# 1 means diabetic type 1
# 2 means diabetic type 2 in phase 11
# 3 means diabetic type 2 in phase 9 but not in phase 11
# 4 means diabetes type 2 in phase 9 and phase 11

def compute_diabetic_outcome(row):

    if row["diagnosed_before_18"] == True:
        return 1
    elif row["only_takes_insulin"] == True:
        return 1
    elif row["high_hba1c"] == 2:
        return 2
    elif row["high_hba1c"] == 3:
        return 3
    elif row["high_hba1c"] == 4:
        return 4
    elif row["takes_oral_tablets"] == 2:
        return 2
    elif row["takes_oral_tablets"] == 3:
        return 3
    elif row["takes_oral_tablets"] == 4:
        return 4
    else:
        return 0

In [ ]:
# Apply defined functions 
df_phase9["diagnosed_before_18"] = df_phase9.apply(diagnosed_before_18, axis=1)
df_phase9["only_takes_insulin"] = df_phase9.apply(only_takes_insulin, axis=1)
df_phase9["high_hba1c"] = df_phase9.apply(high_hba1c, axis=1)
df_phase9["takes_oral_tablets"] = df_phase9.apply(takes_oral_tablets, axis=1)
df_phase9 = df_phase9.drop(df_phase9[(df_phase9["high_hba1c"].isnull()) &
           (df_phase9["diagnosed_before_18"].isnull()) & 
           (df_phase9["only_takes_insulin"].isnull()) & 
           (df_phase9["takes_oral_tablets"].isnull())].index)
df_phase9["diabetic_outcome"] = df_phase9.apply(compute_diabetic_outcome, axis=1)

In [ ]:
# Check the number of participants per diabetic category 
df_phase9["diabetic_outcome"].value_counts()

In [ ]:
 # Drop the features that were created to calculate the target variable 
df_phase9 = df_phase9.drop(["diagnosed_before_18", "only_takes_insulin", "high_hba1c", "takes_oral_tablets"], axis=1)

In [ ]:
# Drop the features that were used to create the features that were used to calculate the target variable 
df_phase9 = df_phase9.drop(columns=target_fts)

In [ ]:
# Save the dataframe as a csv 
df_phase9.to_csv('../Processed datasets/Before splitting/DF_with_target.csv', index=False)